In [1]:
from keras.models import Model
from keras.optimizers import RMSprop
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(rescale=1./255, 
                             brightness_range=[0.8,1.2],
                             rotation_range=20,
                             zoom_range=[0.8,1.2],
                             fill_mode='nearest',
                             validation_split=0.1)

train_datagen = datagen.flow_from_directory('data/train/', seed=42, class_mode='categorical', subset='training', target_size=(512,512))
# prints "60603 images belonging to 101 classes"
val_datagen = datagen.flow_from_directory('data/train/', seed=42, class_mode='categorical', subset='validation', target_size=(512,512)) 
# prints "15150 images belonging to 101 classes"

train_steps = len(train_datagen) #1894
val_steps = len(val_datagen) #474
classes = len(list(train_datagen.class_indices.keys())) #101

Found 68175 images belonging to 102 classes.
Found 7575 images belonging to 102 classes.


In [3]:
model = load_model('trained_top_resnet.h5', compile=True)

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 2048)              23564800  
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 102)              

In [6]:
# for layer in model.get_layer('resnet50v2').layers:
    # print(layer.name)

In [7]:
len(model.trainable_weights)

180

In [8]:
set_trainable = False
for layer in model.get_layer('resnet50v2').layers:
        if layer.name == 'conv5_block1_preact_bn':
            set_trainable = True
        if set_trainable == True:
            layer.trainable = True
        else:
            layer.trainable = False

In [9]:
len(model.trainable_weights)

42

In [10]:
model.compile(loss='categorical_crossentropy',
                      optimizer=RMSprop(lr=0.0001),
                      metrics=['acc','top_k_categorical_accuracy'])

In [11]:
callbacks = [
    ModelCheckpoint(
        filepath='tuned_resnet.h5',
        monitor='val_acc',
        save_best_only=True,
    ),
    EarlyStopping(
        monitor='val_acc',
        patience=2,
    )
]

In [12]:
history = model.fit_generator(
    train_datagen,
    steps_per_epoch=train_steps,
    epochs=50,
    verbose=2,
    validation_data=val_datagen,
    validation_steps=val_steps,
    callbacks=callbacks
)

Epoch 1/50
 - 8951s - loss: 1.7749 - acc: 0.5652 - top_k_categorical_accuracy: 0.8100 - val_loss: 1.1696 - val_acc: 0.7071 - val_top_k_categorical_accuracy: 0.9011
Epoch 2/50
 - 6123s - loss: 1.3890 - acc: 0.6586 - top_k_categorical_accuracy: 0.8708 - val_loss: 1.2801 - val_acc: 0.7296 - val_top_k_categorical_accuracy: 0.9166
Epoch 3/50
 - 5885s - loss: 1.1733 - acc: 0.7130 - top_k_categorical_accuracy: 0.9004 - val_loss: 1.0770 - val_acc: 0.7473 - val_top_k_categorical_accuracy: 0.9213
Epoch 4/50
 - 5875s - loss: 1.0215 - acc: 0.7502 - top_k_categorical_accuracy: 0.9195 - val_loss: 0.8403 - val_acc: 0.7500 - val_top_k_categorical_accuracy: 0.9233
Epoch 5/50
 - 5827s - loss: 0.9060 - acc: 0.7783 - top_k_categorical_accuracy: 0.9313 - val_loss: 0.9589 - val_acc: 0.7633 - val_top_k_categorical_accuracy: 0.9211
Epoch 6/50
 - 5764s - loss: 0.7917 - acc: 0.8049 - top_k_categorical_accuracy: 0.9448 - val_loss: 0.8857 - val_acc: 0.7584 - val_top_k_categorical_accuracy: 0.9251
Epoch 7/50
 - 57